In [1]:
import cv2
from ultralytics import YOLO
import scrcpy.core as scrcpy
import scrcpy.const as const
import tkinter as tk
import customtkinter as ctk
import utils
from PIL import Image
import time
import pickle
from threading import Timer

In [2]:
img_scale = 0.3
update_timer = 1
client = utils.connect_to_device("127.0.0.1:5037")
# Load the YOLOv8 model
model = YOLO("yolo_models/yolo-n-best5.pt")
model.to('cuda:0')
feed_res = (int(client.resolution[0]*img_scale), int(client.resolution[1]*img_scale))

In [ ]:
# class PokemonCatcher(ctk.CTk):
#     def __init__(self,feed_res):
#         super().__init__()
#         self.title("Pokemon Catcher")
#         self.feed_res = feed_res

#         mainframe = ctk.CTkFrame(self)
#         mainframe.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S), padx=30, pady=20)

#         pil_image = Image.new("RGB", feed_res)
#         self.my_image = ctk.CTkImage(light_image=pil_image,dark_image=pil_image, size=feed_res)
#         self.image_label = ctk.CTkLabel(mainframe, text='',image=self.my_image)
#         self.image_label.grid(column=0, row=3, pady=0)  
#         self.screen = None
#         self.last_click_time = 0
#         self.bind('<Button-1>', self.on_touch_down)
#         self.bind('<B1-Motion>', self.on_touch_move)
#         self.bind('<ButtonRelease-1>', self.on_touch_up)

#         self.device_screen_size = client.resolution
#         self.ball_throw_cooldown = 0
#         self.first_pokemon_seen = True  # New flag
#         self.sequence = []
#         self.save_sequence = False

#         with open('ball_throw.pkl', 'rb') as f:
#             self.ball_throw_sequence = pickle.load(f)

#     def on_touch_down(self, event):
#         x, y = self.translate_coordinates(event.x, event.y)
#         client.control.touch(x, y, action=const.ACTION_DOWN)
        
#         # Start a new sequence with this event
#         if self.save_sequence:
#             self.sequence = [(const.ACTION_DOWN, x, y)]

#     def on_touch_move(self, event):
#         x, y = self.translate_coordinates(event.x, event.y)
#         client.control.touch(x, y, action=const.ACTION_MOVE)

#         # Add this event to the sequence
#         if self.save_sequence:
#             self.sequence.append((const.ACTION_MOVE, x, y))

#     def on_touch_up(self, event):
#         x, y = self.translate_coordinates(event.x, event.y)
#         client.control.touch(x, y, action=const.ACTION_UP)

#         # Add this event to the sequence and save the completed sequence to disk
#         if self.save_sequence:
#             self.sequence.append((const.ACTION_UP, x, y))
#             with open('sequence.pkl', 'wb') as f:
#                 pickle.dump(self.sequence, f)
#             print(f"Sequence captured and saved to disk: {self.sequence}")
        
#     def translate_coordinates(self, x, y):
#         window_width, window_height = self.feed_res
#         device_width, device_height = self.device_screen_size
#         translated_x = int(x * device_width / window_width)
#         translated_y = int(y * device_height / window_height)
#         return translated_x, translated_y

#     def perform_ball_throw(self, client):
#         print( time.time() -self.ball_throw_cooldown)
#         # Check if enough time has passed since the last throw
#         if time.time() - self.ball_throw_cooldown < 4:

#             print("Cooldown in progress, skipping throw.")
#             return

#         for action, x, y in self.ball_throw_sequence:
#             client.control.touch(x, y, action=action)

#         print("Performed throw sequence.")
#         self.ball_throw_cooldown = time.time()
#         self.first_pokemon_seen = True


#     def yolo_inference(self, frame):
#         self.screen = frame.copy() 
#         results = model(self.screen,verbose=False)
#         bbox_data = results[0].boxes.data
#         class_names = results[0].names
#         self.pokemon_coords = None

#         if bbox_data.shape[0] > 0:
#             for bbox in bbox_data:
#                 x1, y1, x2, y2, conf, cls = bbox.tolist()
#                 x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

#                 class_name = class_names[int(cls)]
#                 if class_name == "pokemon" or not self.first_pokemon_seen:
#                     # If the class is "pokemon", store the center of the bounding box
#                     self.pokemon_coords = ((x1 + x2) // 2, (y1 + y2) // 2)
#                     if self.first_pokemon_seen:
#                         self.ball_throw_cooldown = time.time()
#                         self.first_pokemon_seen = False
#                     self.perform_ball_throw(client)
   

#                 cv2.rectangle(self.screen, (x1, y1), (x2, y2), (0, 255, 0), 5)
#                 cv2.putText(self.screen, f'{class_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 5)

#     def update_ui(self,client):
#         original_screen = client.last_frame
#         if original_screen is not None:
#             self.yolo_inference(original_screen)
#             resized_image = cv2.resize(self.screen, self.feed_res, interpolation=cv2.INTER_AREA)
#             resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
#             pil_img  = Image.fromarray(resized_image)
#             self.my_image.configure(light_image=pil_img,dark_image=pil_img)

#             if self.pokemon_coords is not None:
#                 self.ititiate_click(self.pokemon_coords, client)
#                 self.pokemon_coords = None  # Reset the pokemon coordinates

#         app.after(update_timer, lambda: self.update_ui(client))

#     def ititiate_click(self, coords, client):
#         current_time = time.time()
#         if current_time - self.last_click_time > 3: 
#             x, y = coords
#             client.control.touch(x, y, action=const.ACTION_DOWN)
#             client.control.touch(x, y, action=const.ACTION_UP)
#             self.last_click_time = current_time 


# if __name__ == "__main__":
#     app = PokemonCatcher(feed_res)
#     app.update_ui(client)
#     app.mainloop()

In [ ]:
class PokemonCatcher(ctk.CTk):
    def __init__(self,feed_res):
        super().__init__()
        self.title("Pokemon Catcher")
        self.feed_res = feed_res

        mainframe = ctk.CTkFrame(self)
        mainframe.grid(column=0, row=0, sticky=(ctk.W, ctk.E, ctk.N, ctk.S), padx=30, pady=20)

        pil_image = Image.new("RGB", feed_res)
        self.my_image = ctk.CTkImage(light_image=pil_image,dark_image=pil_image, size=feed_res)
        self.image_label = ctk.CTkLabel(mainframe, text='',image=self.my_image)
        self.image_label.grid(column=0, row=3, pady=0)  
        self.screen = None
        self.last_click_time = 0
        self.bind('<Button-1>', self.on_touch_down)
        self.bind('<B1-Motion>', self.on_touch_move)
        self.bind('<ButtonRelease-1>', self.on_touch_up)

        self.device_screen_size = client.resolution
        self.ball_throw_cooldown = 0
        self.first_pokemon_seen = True  # New flag
        self.sequence = []
        self.save_sequence = False
        self.class_colors = {
            'pokestop-unspun': (0, 255, 0),  # green
            'pokestop-spun': (255, 0, 0),    # red
            'pokemon': (0, 0, 255),          # blue
            'gym': (255, 255, 0),            # cyan
            'pokestop-inactive': (0, 255, 255),  # yellow
        }

        with open('ball_throw.pkl', 'rb') as f:
            self.ball_throw_sequence = pickle.load(f)

    def on_touch_down(self, event):
        x, y = self.translate_coordinates(event.x, event.y)
        client.control.touch(x, y, action=const.ACTION_DOWN)
        
        # Start a new sequence with this event
        if self.save_sequence:
            self.sequence = [(const.ACTION_DOWN, x, y)]

    def on_touch_move(self, event):
        x, y = self.translate_coordinates(event.x, event.y)
        client.control.touch(x, y, action=const.ACTION_MOVE)

        # Add this event to the sequence
        if self.save_sequence:
            self.sequence.append((const.ACTION_MOVE, x, y))

    def on_touch_up(self, event):
        x, y = self.translate_coordinates(event.x, event.y)
        client.control.touch(x, y, action=const.ACTION_UP)

        # Add this event to the sequence and save the completed sequence to disk
        if self.save_sequence:
            self.sequence.append((const.ACTION_UP, x, y))
            with open('sequence.pkl', 'wb') as f:
                pickle.dump(self.sequence, f)
            print(f"Sequence captured and saved to disk: {self.sequence}")
        
    def translate_coordinates(self, x, y):
        window_width, window_height = self.feed_res
        device_width, device_height = self.device_screen_size
        translated_x = int(x * device_width / window_width)
        translated_y = int(y * device_height / window_height)
        return translated_x, translated_y

    def perform_ball_throw(self, client):
        # Check if enough time has passed since the last throw
        if time.time() - self.ball_throw_cooldown < 4:
            print("Cooldown in progress, skipping throw.")
            return

        # Schedule the throw to happen after a delay
        Timer(4, self.do_ball_throw, [client]).start()

        print("Scheduled throw sequence.")
        self.ball_throw_cooldown = time.time()
        self.first_pokemon_seen = True

    def do_ball_throw(self, client):
        # after the delay
        for action, x, y in self.ball_throw_sequence:
            client.control.touch(x, y, action=action)

        print("Performed throw sequence.")

    def yolo_inference(self, frame):
        self.screen = frame.copy() 
        results = model(self.screen,verbose=False)
        bbox_data = results[0].boxes.data
        class_names = results[0].names
        self.pokemon_coords = None

        if bbox_data.shape[0] > 0:
            for bbox in bbox_data:
                x1, y1, x2, y2, conf, cls = bbox.tolist()
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

                class_name = class_names[int(cls)]
                color = self.class_colors.get(class_name, (255, 255, 255))  # default to white
                if class_name == "pokemon" or not self.first_pokemon_seen:
                    # If the class is "pokemon", store the center of the bounding box
                    self.pokemon_coords = ((x1 + x2) // 2, (y1 + y2) // 2)
                    if self.first_pokemon_seen:
                        self.ball_throw_cooldown = time.time()
                        self.first_pokemon_seen = False
                    # self.perform_ball_throw(client)
   
                cv2.rectangle(self.screen, (x1, y1), (x2, y2), color, 5)
                cv2.putText(self.screen, f'{class_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 5)

    def update_ui(self,client):
        original_screen = client.last_frame
        if original_screen is not None:
            self.yolo_inference(original_screen)
            resized_image = cv2.resize(self.screen, self.feed_res, interpolation=cv2.INTER_AREA)
            resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
            pil_img  = Image.fromarray(resized_image)
            self.my_image.configure(light_image=pil_img,dark_image=pil_img)

            # if self.pokemon_coords is not None:
            #     self.ititiate_click(self.pokemon_coords, client)
            #     self.pokemon_coords = None  # Reset the pokemon coordinates

        app.after(update_timer, lambda: self.update_ui(client))

    def ititiate_click(self, coords, client):
        current_time = time.time()
        if current_time - self.last_click_time > 3: 
            x, y = coords
            client.control.touch(x, y, action=const.ACTION_DOWN)
            client.control.touch(x, y, action=const.ACTION_UP)
            self.last_click_time = current_time 


if __name__ == "__main__":
    app = PokemonCatcher(feed_res)
    app.update_ui(client)
    app.mainloop()